<a href="https://colab.research.google.com/github/mrkct/cuda-raytracer/blob/master/Relazione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
!rm -rf cuda-raytracer
!git clone https://github.com/mrkct/cuda-raytracer.git
%cd cuda-raytracer
!make all

# Raytracer accelerato via *CUDA*

Il raytracing è una tecnica utilizzata in computer grafica per il rendering di immagini realistiche. L'idea alla base del raytracing è quella di proiettare dei raggi di luce a partire dall'occhio dello spettatore e, simulando il comportamento dei raggi che interagiscono con gli oggetti nella scena, determinare il colore di un pixel dell'immagine finale. Questa tecnica è raramente utilizzata nella computer grafica in tempo reale a causa della sua pesantezza computazionale perchè richiede di simulare un numero significativo di raggi; almeno uno per ogni pixel dell'immagine finale.
Il calcolo di questi raggi è completamente indipendente tra di loro; questo rende il raytracing rende un candidato ideale alla parallelizzazione.

Il raytracer implementato in questo report è basato su quello  del libro *Raytracing in One Weekend*[1], dove viene implementato in una versione single-thread su CPU. In aggiunta alle feature del libro ho implementato anche il texturing degli oggetti e l'export delle immagini in formato `png`; il video qui sotto è stato prodotto dal mio raytracer generando le immagini dei singoli fotogrammi ed assemblandoli in un video utilizzando `ffmpeg`.**bold text**

In [ ]:
from IPython.display import Video
Video("demo.mp4")

# Algoritmo per il raytracing

Alla base del raytracing sta il concetto di *raggio*; un raggio è semplicemente una retta in uno spazio tridimensionale, dunque possiamo esprimere i punti su di essa con la semplice formula $P(t) = A + B \cdot t$.

Definiremo questi raggi tramite due punti per cui passano: il primo punto è l'occhio dello spettatore (una costante per tutti i raggi iniziali), l'altro è un punto sulla *vista*. Questa vista è una proiezione nello spazio 3D dell'immagine finale che vorremo generare: dato un punto di coordinate $(x, y)$ nell'immagine finale allora il suo corrispondente nello spazio 3D sul piano sarà dato da

$$
  \frac{x}{ImageWidth} \cdot ViewWidth \cdot H + \frac{y}{ImageHeight} \cdot ViewHeight \cdot V + LowerLeftCorner
$$

In questa formula $H$ e $V$ sono due vettori unitari che puntano nelle direzioni per muoversi orizzontalmente e verticalmente sul piano.

La dimensione e posizione del piano rispetto alla telecamera ha effeti interessanti sull'immagine finale: tenendo fissa la dimensione del piano ma allontanandolo dallo spettatore provoca un avvicinamento sempre maggiore dei raggi proiettati sul piano che finiscono per concentrarsi un'area più piccola ma in modo più denso: questo è come funziona lo zoom di una telecamera. La distanza di questo piano dalla telecamera è un parametro detto *lunghezza focale*.

    **IMMAGINI DEL CAMBIO DI FOCAL LENGTH**

Invece, tenendo fissa la distanza tra telecamera e vista ma variando la dimensione di quest'ultima blabla.

Un raytracer, per ogni pixel dell'immagine, proietta un raggio con associato un colore iniziale ed osserva il colore finale del raggio dopo che questo va a collidere con eventuali oggetti nella sua traiettoria dalla telecamera al piano. Il materiale di cui sono fatti gli oggetti con cui un raggio va a collidere determina il modo in cui il colore del raggio viene modificato: per esempio un materiale opaco assorbe una parte di luminosità del raggio.

Il seguente pseudocodice mostra il "cuore" di un raytracer, seppur nasconda alcuni aspetti importanti come un modo per trovare l'intersezione tra raggio e oggetti oppure la *funzione di dispersione di colore* dei materiali: il motivo è semplicemente che questi aspetti dipendono dalla scena che vogliamo renderizzare.

```c
for (int x = 0; x < ImageWidth; x++) {
  for (int y = 0; y < ImageHeight; y++) {
    Sia `p` il punto sul piano corrispondente al pixel (x, y)
    Sia `r` una retta che passa per i punti `CameraPosition` e `p`
    Sia `o` il primo oggetto che interseca la retta `r` partendo da `CameraPosition`
      in avanti
    
    if (la retta `r` non ha intersecato alcun oggetto) then {
      Il pixel (x, y) ha colore dello sfondo
    } else {
      Sia `m` il materiale di cui è composto `o`
      Sia `c` il colore ottenuto applicando la funzione di dispersione di colore
        del materiale `m` al colore dello sfondo
      Il pixel (x, y) ha colore `c`
    }
  }
}
```

## Calcolo dell'intersezione tra un raggio ed una sfera

Per trovare se la retta `r` interseca con qualche oggetto una possibile soluzione è quella di iterare su tutti gli oggetti della scena e, utilizzando qualche formula geometrica, calcolare il punto di intersezione tra l'oggetto e la retta (se esiste). L'oggetto con il punto di intersezione con distanza minore dalla telecamera è quello il cui materiale deve essere usato per determinare il colore del raggio.

Per esempio se volessimo calcolare l'intersezione tra una retta ed una sfera di raggio $r$ centrata in $C$ potremmo osservare che i punti su una sfera sono descritti dall'equazione:

$$
  (x - C_x)^2 + (y - C_y)^2 + (z - C_z)^2 = r^2
$$

Che possiamo scrivere anche in questo modo:

$$
  r^2 = (P - C) \cdot (P - C)
$$

I punti del nostro raggio sono descritti dalla funzione $P(t)=A + t \cdot b$, vogliamo dunque trovare se esistono dei punti su $P(t)$ che soddisfano anche l'equazione. Sostituiamo dunque a $P$ l'equazione della nostra retta e, dopo un po' di semplificazioni matematiche, otteniamo una equazione di secondo grado:

$$
  (A + t \cdot B - C) \cdot (A + t \cdot B - C) = r^2
$$
$$
  t \cdot 2b^2 + t \cdot 2b (A-C) + (A-C) \cdot (A-C) - r^2 = 0
$$

In questa equazione l'incognita $t$ sono i punti di intersezione, è possibile risolverla con la classica formula $\frac{-b +- \sqrt{b^2 - 4ac}}{2a}$. In base al numero di soluzioni trovate possiamo capire se il raggio non interseca la sfera (zero soluzioni trovate, discriminante negativo), se la retta passa tangente per un solo punto (una sola soluzione, discriminante uguale a zero) oppure se la retta attraversa la sfera e dunque ci sono due punti di intersezione; in quest'ultimo caso sceglieremo la soluzione con distanza minore dalla telecamera.

    Immagine delle rette che attraversano il cerchio

## Calcolo della funzione di dispersione del colore di un materiale opaco

Un materiale opaco prende il suo colore mischiando il suo colore innato (detto *albedo*) a quello ottenuto dai materiali riflessi su di esso. Il materiale opaco che andiamo a simulare è detto *Lambertiano* e la sua riflettenza è pari in qualunque punto e direzione. Un modo molto semplice per simulare il colore di un raggio che interseca un materiale Lambertiano è quello di prendere l'albedo del materiale (rappresentato come un vettore a 3 dimensioni di valori $RBG$ tra $[0, 1]$) e di moltiplicarlo componente per componente con il colore di un raggio proiettato dal punto di intersezione e il vettore normale della superficie.

    Immagine della sfera e dei raggi che rimbalzano tra sfera e pavimento

Per esempio, se determiniamo che il nostro raggio colpisce una sfera nella posizione $P$ e questa sfera è composta di un materiale Lambertiano con albedo rosso $(0.7, 0, 0)$, allora per calcolare il colore del raggio dovremo prima determinare ricorsivamente il colore di un raggio che parte da $P$ ed ha come direzione il vettore normale alla curva e poi moltiplicare questo colore per l'albedo del materiale, in questo caso $(0.7, 0, 0)$.

Per determinare il vettore normale alla superficie di una sfera basta sottrarre al punto $P$ il punto origine della sfera e normalizzare il vettore.

# Accelerazione tramite GPU

Per accelerare via CUDA questo processo ho diviso l'immagine finale in blocchi quadrati di lato $8$, ogni thread si occupa di un singolo pixel dell'immagine finale e dunque proietta il raggio associato a quel pixel. La scelta di questo valore è data dal voler ottenere una dimensione dei blocchi multipla di $32$ per utilizzare al massimo ogni warp, ma anche di voler ridurre al minimo la dimensione dei blocchi per ridurre al minimo la divergenza nel warp.

Il calcolo di due raggi può essere significativamente diverso in base al materiale che intersecano: determinare il colore di un raggio che non colpisce nulla ha un costo computazionale nullo, invece determinare il colore di un raggio che colpisce un materiale altamente riflettente implica dover calcolare altri raggi riflessi. Questo vuol dire che due raggi proiettati da thread all'interno dello stesso blocco possono dover eseguire algoritmi estremamente diversi, il che implica un problema significativo di divergenza nel warp.

Ridurre la dimensione dei blocchi non aiuta a ridurre la divergenza nel calcolo dei singoli raggi, ma nella pratica raggi vicini vanno a collidere con lo stesso materiale e dunque eseguono lo stesso algoritmo, **il limite tra cazzata e genio è molto sottile qui, bisogna stare attenti a quello che si scrive**

    TABELLA DELLE PERFORMANCE VARIANDO blockSize  
    mostra (4, 4), (8, 4), (8, 8), (16, 8), (16, 16)

I thread scrivono il colore finale del loro pixel su un framebuffer allocato utilizzando la managed memory: il motivo di questa scelta è il fatto che la memoria del framebuffer viene spostata tra host e device solamente all'inizio ed al termine del rendering di un fotogramma, dunque seppur non sia complicato gestire questi spostamenti manualmente non ho trovato alcuna differenza tra i due modi.

Prima di iniziare il rendering vengono caricate in *constant memory* gli elementi della scena: questi includono gli oggetti e i loro materiali associati. Infine i valori *RGB* delle texture utilizzate dagli oggetti sono salvati in *texture memory*.

## Confronto con implementazione CPU

La versione del raytracer accelerato via CUDA offre un notevole speedup rispetto a quelle implementate su CPU, sia che utilizzino un singolo thread sia molteplici.

Le implementazioni CPU-based confrontate sono quella ufficiale del libro[2], utilizzante un solo thread, ed una mia implementazione[3] scritta in passato che utilizza un numero di thread pari al numero di core della CPU.
Questo confronto non pretende di essere un confronto particolarmente corretto nei confronti di quest'ultime due implementazioni dato che quella del libro esiste solamente a scopo didattico, e dunque non si concentra sulle performance, mentre invece la mia implementazione CPU-based è scritta nel linguaggio di programmazione Rust che, seppur comunque nello stesso range di prestazioni del C++, è comunque un linguaggio diverso.

Le implementazioni CUDA e C++ sono state compilate utilizzando il flag `-O3`, la implementazione in Rust è stata compilata con il flag `--build release`: queste flag dovrebbero aver abilitato il massimo livello di ottimizzazione.

Il confronto è stato fatto eseguendo ogni programma 10 volte, una dopo l'altra, renderizzando una immagine di risoluzione 8k (7680 x 4320 pixels) della scena mostrata nel video ad inizio relazione. La macchina usata nei test è una macchina offerta da *Colab* con le seguenti specifiche:

    CPU: Intel X
    RAM: Y GB
    GPU: NVidia Z
    OS: Linux 64bit

I tempi misurati sono solamente quelli per il rendering dell'immagine, definito come il momento in cui il framebuffer in memoria contiene i valori RGB di tutti i pixel. Queste misurazioni dunque escludono il tempo speso per preparare la scena da renderizzare e per scrivere l'immagine finale su disco.

I risultati finali, mostrati nella tabella qui sottostante, mostrano uno speedup schiacciante rispetto alle versioni CPU-based; la versione CUDA è in media 80000000000% più veloce di quella CPU-based a single thread e in media 60000000000000% più veloce di quella multi-thread.

| Test 	| 1 	| 2 	| 3 	| 4 	| 5 	| 6 	| 7 	| 8 	| 9 	| 10 	| Media 	|
|---------------	|---	|---	|---	|---	|---	|---	|---	|---	|---	|----	|-------	|
| CUDA          	|   	|   	|   	|   	|   	|   	|   	|   	|   	|    	|       	|
| 1-Thread      	|   	|   	|   	|   	|   	|   	|   	|   	|   	|    	|       	|
| 8-Thread      	|   	|   	|   	|   	|   	|   	|   	|   	|   	|    	|       	|




## Profiling delle prestazioni

**parlo un po' dei risultati di nv-nsight-cu-cli e dei bottleneck**

# Showcase di immagini generate


    Inserire immagine delle tre sfere con i materiali

    Inserire immagine delle tante sfere random dal libro

    Inserire immagine delle sfere con texture pianeti

# Bibliografia
[1]: [_Ray Tracing in One Weekend_](https://raytracing.github.io/books/RayTracingInOneWeekend.html), Peter Shirley, 2020  
[2]: [_Ray Tracing in One Weekend Source Code_](https://github.com/RayTracing/raytracing.github.io/tree/master/src/InOneWeekend), Peter Shirley, 2020  
[3]: [Sphere Point Picking](https://mathworld.wolfram.com/SpherePointPicking.html), Eric W. Weisstein, Wolfram MathWorld